In [13]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import sys
sys.path.append('../')

In [15]:
import os
import h2o
import pandas as pd

from h2o.automl import H2OAutoML
from utilities import save_model as save

home_dir = os.path.join(os.path.expanduser('~'), 'Documents/TDT4173-ML')
train_data_path = os.path.join(home_dir, 'data_feature_engineering')
target_data_path = os.path.join(home_dir, 'data_processed')

In [16]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 mins 28 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_stinky_l3b4cy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.896 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [17]:
train = pd.read_csv(os.path.join(train_data_path, 'summed_no_snow_train.csv'))
target = pd.read_csv(os.path.join(target_data_path, 'target.csv'))

train.set_index('date_forecast', inplace=True)
target.set_index('time', inplace=True)

In [18]:
X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [19]:
autoML = H2OAutoML(max_models=30, seed=7)
autoML.train(x=X, y=y, training_frame=X_train)

AutoML progress: |


14:07:19.154: XGBoost_1_AutoML_2_20231102_140719 [XGBoost def_2] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_1_AutoML_2_20231102_140719_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


█████████
14:08:17.661: XGBoost_2_AutoML_2_20231102_140719 [XGBoost def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_2_AutoML_2_20231102_140719_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


███████
14:12:22.428: XGBoost_3_AutoML_2_20231102_140719 [XGBoost def_3] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_3_AutoML_2_20231102_140719_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


████████████

key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/30
# GBM base models (used / total),8/13
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/14
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [20]:
best_model = autoML.leader
save.save_model_to_local_disk(best_model, '01E_h2o_summed_no_snow')